<a href="https://colab.research.google.com/github/FabriceBeaumont/2-MA-INF-4316-GRL/blob/main/Exercises/Exercise9/GRL_test_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 9: MA-INF4316 Graph Representation Learning

- Wintersemester 2021/2022
- Exam: 0
- Date 2022-01-17
- Examiner: Dr. Pascal Welke

### To be filled by the student
- Name:                  Fabrice
- Given Name:            Beaumont
- Matriculation number:  2747609
- Course of Studies:     M.Sc. (Uni Bonn)

(Please enter your data here)

## Imports

In [2]:
!pip install grakel

     |████████████████████████████████| 1.6 MB 5.4 MB/s 
     |████████████████████████████████| 154 kB 49.5 MB/s 


In [4]:
!pip install igraph

     |████████████████████████████████| 3.1 MB 5.4 MB/s 


In [5]:
import grakel
from grakel.datasets import fetch_dataset
import igraph
import numpy as np
import matplotlib.pyplot as plt

# Task 1 - Intersection Kernels

Let $X$ be a set. Show that the intersection kernel on histograms $X\in\mathbb{N}^{\mathcal{X}}$:
$$ k(A, B) = \sum\limits_{x\in\mathcal{X}} \min\big( \Gamma_x(A), \Gamma_x(B) \big)$$
is a kernel. Recall from Lecture 8 that $\Gamma_x (A) \in\mathbb{N}^{\mathcal{X}}$  gives the number of times that $x$ is contained in $A\in\mathbb{N}^{\mathcal{X}}$.

**Hint:** Try to create an (infinite dimensional) embedding $\Phi:\mathbb{N}^{\mathcal{X}}:\{0,1\}^{\infty}$
explicitly.

### Solution of task 1:

I will assume that $\mathcal{X}$ is of finite size and we can write $\mathcal{X} = \{x_0,\dots, x_{|\mathcal{X}|-1} \}$.

In words, the intersetion kernel $k$ sums up the least occurrences in both sets of all elements. The minimum is zero if and only if the current element is not present in at least one of the sets. Thus the sum is zero, if and only if the sets are disjoint. When constructing $\Phi$, we thus require a mapping, such that the resulting vectors in $\{0,1\}^\infty$ are orthogonal, if the sets are disjoint.

Since $\Phi$ needs to measure the number of occurrences of elements in the given set, the following mapping comes to mind:

$$\forall A\in\mathbb{N}^{\mathcal{X}}:\quad \Phi(A) := \phi^A \in\{0,1\}^\infty$$
where
$$ \phi^A_{n|\mathcal{X}|+i} = \begin{cases}
1 & \text{if } n \le \Gamma_{x_i}(A) & \text{($x_i$ occurs at least $n$ times in $A$)}\\
0 & \text{otherwise} & \text{($x_i$ occurs less than $n$ times in $A$ - possibly not at all)}
\end{cases} $$

Note that if $n=\Gamma_{x_i}(A) \le \Gamma_{x_i}(B) = n^\prime$, then it is:
\begin{align*}
    \forall m\le n&: \qquad &\phi^A_{m|\mathcal{X}|+i} = 1 = \phi^B_{m|\mathcal{X}|+i}\\
    \forall n<m\le n^\prime&: &\phi^A_{m|\mathcal{X}|+i} = 0 \neq 1 = \phi^B_{m|\mathcal{X}|+i}\\
    \forall n^\prime < m&: &\phi^A_{m|\mathcal{X}|+i} = 0 = \phi^B_{m|\mathcal{X}|+i}\\
\end{align*}

Thus for all elements $x\in\mathcal{X}$, $\Phi(A)$ and $\Phi(B)$ have exaclty $\min\big(\Gamma_x(A),\Gamma_x(B)\big)$ common non-zero entries - and their scalar product adds all these minimums for all elements to the desired sum:

$$\implies k(A,B) = \langle \Phi(A), \Phi(B) \rangle $$

# Task 2 - Diagonal Dominance

Using grakel (or any other means you like), load the dataset `DHFR` from
the TU Dortmund dataset collection. Choose a random 90/10 train/test split and compute the Weisfeiler Lehman graph kernel for $k = 3$ iterations and train a support vector machine.
1. Print the test accuracy of a SVC classifier trained on the test split.
2. Also, print the number of support vectors for each class.

## Task 2.1 - Creating Diagonally Dominant Kernels

Now, to investigate the behavior of the SVM classifier for more and more diagonally-dominant kernels, recall, that the sum of two kernel functions is a kernel function, and that the dirac delta kernel is a kernel.

During the previous subtask, you should have computed a Gram matrix. You may create a (more) diagonally dominant Gram matrix, by adding a constant value to all entries of the diagonal of that Gram matrix.

1. Compute the maximum value $K_m ax$ in the Gram matrix of your Weisfeiler Lehman kernel and output it.
2. For i in $\{0, 0.01, 0.1, 1, 10, 100\}$, add $i\cdot K_m ax$ to all diagonal entries of your Gram matrix and report the test accuracy of a SVM trained with
this Gram matrix, as well as the number of support vectors for each
class.
3. What do you observe?

**Hint:** If you are working with grakels methods, you might wonder how “adding to the diagonal” works for the asymmetric test “Gram” matrix 

    wl_kernel.transform(G_test)

Here, you don’t need to do anything, as this matrix contains kernel values between graphs from train and test, which means that there will be no diagonal element of the overall Gram matrix of the DHFR dataset present.

## Task 2.2 - Additional Considerations

The above approach results in a positive semidefinite Gram matrix on the whole dataset, as well as, any subset of it. However, there are some severe issues (apart from the accuracy drop that you should have observed) with such an approach.
1. Is the kernel that you obtain for any $i > 0$ invariant under isomorphism?

# Task 3 - Graph Edit Distance Non-Kernels

Recall from Lecture 9 that the function
$$ f(x,y) = \exp\big(-\gamma d(x,y)\big)$$
can create a kernel from some distance functions $d$. This is the case, for example, for the Euclidean distance.
Show that this does not work if $d$ is the graph edit distance. That is, show that the graph edit distance $\hat{d}$ defined by the edit cost function $c$ on $\Sigma_\epsilon = \{a, b, c, \epsilon\}$ given by

.   || a   | b   | c   | $\epsilon$
--- || --- | --- | --- | ---
b   || 0   | 1   | 16  | 32         
c   || 0   | 1   | 16  | 32         
$\epsilon$ || 0 | 1 | 16 | 32         

There exist three (rather simple) graphs such that the Gram matrix these three graphs for $\exp\big(−\gamma d(x,y)\big)$ is not positive semidefinite.

**Hint:** Check out numpys `numpy.linalg.eigvalsh` and `numpy.exp` functions.

# Task 4 - Optimal Assignment Kernel

Show that the construction
$$ [H^k (X)]_v = \Big(w(v) − w\big(parent(v)\big)\Big) × |X_v| $$
on Slide 20 of Lecture 9 defines a histogram for any hierarchy $H$.

# Task 5 - Weisfeiler Lehman Optimal Assignment Kernel

## Task 5.1
Implement the Weisfeiler Lehman Optimal Assignment Kernel (using the formulation on Slide 20 of Lecture 9).

## Task 5.2
Apply the Weisfeiler Lehman Optimal Assignment Kernel on the DHFR
dataset for a depth of $k = 3$. Compare its classification performance to that of the ordinary Weisfeiler Lehman subtree kernel of the same depth $k = 3$.

**Note:** You can compare the results of your implementation to that of the [grakel implementation](https://ysig.github.io/GraKeL/0.1a8/kernels/weisfeiler_lehman_optimal_assignment.html) of the Weisfeiler Lehman Optimal Assignment kernel, or just straight away apply this variant, if you are not interested in implementing it yourself.